In [ ]:
import gc
import os
from pathlib import Path
import random
import sys

from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import scipy as sp


import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import pickle
from pathlib import Path

import cv2
import pandas as pd
from tqdm import tqdm

# --- setup ---
pd.set_option('max_columns', 50)


In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
train = pd.read_csv("../input/vinbigdata-chest-xray-resized-png-256x256/train_meta.csv")

****Check torch version

In [ ]:
import torch
torch.__version__

Install detectron2

In [ ]:
!pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html

In [ ]:
from detectron2.structures import BoxMode #boxmode is used to select the style of bounding boxes

In [ ]:
def get_vinbigdata_dicts(
    imgdir: Path, train: pd.DataFrame, use_cache: bool = True, debug: bool = True,
):
    
        train_meta = pd.read_csv(imgdir / "train_meta.csv")
        image_id = train_meta.loc[0, "image_id"]
        image_path = str(imgdir / "train" / f"{image_id}.png")
        image = cv2.imread(image_path)
        resized_height, resized_width, ch = image.shape
        print(f"image shape: {image.shape}")

        dataset_dicts = []
        for index, train_meta_row in tqdm(train_meta.iterrows(), total=len(train_meta)):
            record = {}

            image_id, height, width = train_meta_row.values
            filename = str(imgdir / "train" / f"{image_id}.png")
            record["file_name"] = filename
            record["image_id"] = index
            record["height"] = resized_height
            record["width"] = resized_width
            objs = []
            for index2, row in train.query("image_id == @image_id").iterrows():
                class_id = row["class_id"]
                if class_id == 14:
                    pass
                else:
                    h_ratio = resized_height / height

                    w_ratio = resized_width / width
                    bbox_resized = [
                        int(row["x_min"]) * w_ratio,
                        int(row["y_min"]) * h_ratio,
                        int(row["x_max"]) * w_ratio,
                        int(row["y_max"]) * h_ratio,
                    ]
                    obj = {
                        "bbox": bbox_resized,
                        "bbox_mode": BoxMode.XYXY_ABS,
                        "category_id": class_id,
                    }
                    objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
            
        return dataset_dicts





In [ ]:
def get_vinbigdata_dicts_test(
    imgdir: Path, test_meta: pd.DataFrame, use_cache: bool = True, debug: bool = True,
):
        # test_meta = pd.read_csv(imgdir / "test_meta.csv")

        # Load 1 image to get image size.
        image_id = test_meta.loc[0, "image_id"]
        image_path = str(imgdir / "test" / f"{image_id}.png")
        image = cv2.imread(image_path)
        resized_height, resized_width, ch = image.shape
        print(f"image shape: {image.shape}")

        dataset_dicts = []
        for index, test_meta_row in tqdm(test_meta.iterrows(), total=len(test_meta)):
            record = {}

            image_id, height, width = test_meta_row.values
            filename = str(imgdir / "test" / f"{image_id}.png")
            record["file_name"] = filename
            record["image_id"] = index
            record["height"] = resized_height
            record["width"] = resized_width
            # objs = []
            # record["annotations"] = objs
            dataset_dicts.append(record)
        return dataset_dicts

In [ ]:
# --- utils ---
from pathlib import Path
from typing import Any, Union

import yaml


def save_yaml(filepath: Union[str, Path], content: Any, width: int = 120):
    with open(filepath, "w") as f:
        yaml.dump(content, f, width=width)


def load_yaml(filepath: Union[str, Path]) -> Any:
    with open(filepath, "r") as f:
        content = yaml.full_load(f)
    return content


In [ ]:
# --- configs ---
thing_classes = [
    "Aortic enlargement",
    "Atelectasis",
    "Calcification",
    "Cardiomegaly",
    "Consolidation",
    "ILD",
    "Infiltration",
    "Lung Opacity",
    "Nodule/Mass",
    "Other lesion",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary fibrosis"
]
category_name_to_id = {class_name: index for index, class_name in enumerate(thing_classes)}


In [ ]:
from dataclasses import dataclass
from typing import Dict
@dataclass
class Flags:
    # General
    debug: bool = True
    outdir: str = "results/det"

    # Data config
    imgdir_name: str = "vinbigdata-chest-xray-resized-png-1024x1024"
    iter: int = 100000
    ims_per_batch: int = 2 
    num_workers: int = 4
    base_lr: float = 0.00025
    roi_batch_size_per_image: int = 256

    def update(self, param_dict: Dict) -> "Flags":
        # Overwrite by `param_dict`
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f"[ERROR] Unexpected key for flag = {key}")
            setattr(self, key, value)
        return self


In [ ]:

import argparse
import dataclasses
import json
import os
import pickle
import random
import sys
from dataclasses import dataclass
from distutils.util import strtobool
from pathlib import Path

import cv2
import detectron2
import numpy as np
import pandas as pd
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator
from detectron2.structures import BoxMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from tqdm import tqdm


setup_logger()


class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        # return COCOEvaluator(dataset_name, cfg, True, output_folder)
        return COCOEvaluator(dataset_name, ("bbox",), False, output_dir=output_folder)

In [ ]:
flags_dict = {
    "debug": True,
    "outdir": "results/debug", 
    "imgdir_name": "vinbigdata-chest-xray-resized-png-1024x1024",
    "iter": 100000,  # debug, small value should be set.
    "roi_batch_size_per_image": 256  # faster, and good enough for this toy dataset (default: 512)
}

In [ ]:
# args = parse()
print("torch", torch.__version__)
flags = Flags().update(flags_dict)
print("flags", flags)
debug = flags.debug
outdir = Path(flags.outdir)
os.makedirs(str(outdir), exist_ok=True)
flags_dict = dataclasses.asdict(flags)
save_yaml(outdir / "flags.yaml", flags_dict)

# --- Read data ---
inputdir = Path("/kaggle/input")
datadir = inputdir / "vinbigdata-chest-xray-abnormalities-detection"
imgdir = inputdir / flags.imgdir_name

# Read in the data CSV files
train = pd.read_csv(datadir / "train.csv")
# sample_submission = pd.read_csv(datadir / 'sample_submission.csv')

DatasetCatalog.register(
    "vinbigdata_train", lambda: get_vinbigdata_dicts(imgdir, train, debug=False)
)
MetadataCatalog.get("vinbigdata_train").set(thing_classes=thing_classes)

In [ ]:
dataset_dicts = get_vinbigdata_dicts(imgdir, train,debug = False)

In [ ]:
len(dataset_dicts)

In [ ]:
'''dataset_dicts = get_vinbigdata_dicts_test(imgdir, test_meta, debug=False)'''

In [ ]:
anomaly_image_ids = train.query("class_id != 14")["image_id"].unique()
anomaly_image_ids

In [ ]:
train_meta = pd.read_csv(imgdir/"train_meta.csv")

anomaly_inds = np.argwhere(train_meta["image_id"].isin(anomaly_image_ids).values)[:, 0]
print(anomaly_inds.shape)
print(anomaly_inds)
r = anomaly_inds[:9]
r.shape
r

In [ ]:
# Visualize data...
anomaly_image_ids = train.query("class_id != 14")["image_id"].unique()
train_meta = pd.read_csv(imgdir/"train_meta.csv")
anomaly_inds = np.argwhere(train_meta["image_id"].isin(anomaly_image_ids).values)[:, 0]

vinbigdata_metadata = MetadataCatalog.get("vinbigdata_train")

cols = 3
rows = 3
fig, axes = plt.subplots(rows, cols, figsize=(18, 18))
axes = axes.flatten()

for index, anom_ind in enumerate(anomaly_inds[:cols * rows]):
    ax = axes[index]
    # print(anom_ind)
    d = dataset_dicts[anom_ind]
    #print(d)
    img = cv2.imread(d["file_name"])
    #r = img
    #print(r.shape)
    visualizer = Visualizer(img, metadata=vinbigdata_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    # cv2_imshow(out.get_image()[:, :, ::-1])
    #cv2.imwrite(str(outdir / f"vinbigdata{index}.jpg"), out.get_image()[:, :, ::-1])
    ax.imshow(out.get_image()[:, :, ::-1])
    ax.set_title(f"{anom_ind}: image_id {anomaly_image_ids[index]}")

In [ ]:
cfg = get_cfg()
original_output_dir = cfg.OUTPUT_DIR
cfg.OUTPUT_DIR = str(outdir)
print(f"cfg.OUTPUT_DIR {original_output_dir} -> {cfg.OUTPUT_DIR}")

config_name = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(config_name))
cfg.DATASETS.TRAIN = ("vinbigdata_train",)
cfg.DATASETS.TEST = ()
# cfg.DATASETS.TEST = ("vinbigdata_train",)
# cfg.TEST.EVAL_PERIOD = 50
cfg.DATALOADER.NUM_WORKERS = flags.num_workers
# Let training initialize from model zoo
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_name)
cfg.SOLVER.IMS_PER_BATCH = flags.ims_per_batch
cfg.SOLVER.BASE_LR = flags.base_lr  # pick a good LR
cfg.SOLVER.MAX_ITER = flags.iter
cfg.SOLVER.CHECKPOINT_PERIOD = 11000 # Small value=Frequent save need a lot of storage.
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = flags.roi_batch_size_per_image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(thing_classes)
# NOTE: this config means the number of classes,
# but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
test_meta = pd.read_csv("../input/vinbigdata-testmeta/test_meta.csv")
sample_submission = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv")

In [ ]:
inputdir = Path("/kaggle/input")
traineddir = inputdir /"results/debug"

In [ ]:
cfg.MODEL.WEIGHTS = str("./results/debug/model_final.pth")
print("Original thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)  # 0.05
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0  # set a custom testing threshold
print("Changed  thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)
cfg.DATASETS.TEST = ("vinbigdata_test",)
predictor = DefaultPredictor(cfg)

In [ ]:

DatasetCatalog.register(
    "vinbigdata_test", lambda: get_vinbigdata_dicts_test(imgdir, test_meta)
)
MetadataCatalog.get("vinbigdata_test").set(thing_classes=thing_classes)
metadata = MetadataCatalog.get("vinbigdata_test")
dataset_dicts = get_vinbigdata_dicts_test(imgdir, test_meta)

In [ ]:
dataset_dict = get_vinbigdata_dicts_test(imgdir, test_meta)

In [ ]:
import math

In [ ]:

results_list = []
index = 0
batch_size = 4

In [ ]:
from math import ceil

In [ ]:
from math import ceil
from typing import Any, Dict, List

import cv2
import detectron2
import numpy as np
from numpy import ndarray
import pandas as pd
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode, Visualizer
from tqdm import tqdm

In [ ]:
def format_pred(labels: ndarray, boxes: ndarray, scores: ndarray) -> str:
    pred_strings = []
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.int64)
        pred_strings.append(f"{label} {score} {xmin} {ymin} {xmax} {ymax}")
    return " ".join(pred_strings)


def predict_batch(predictor: DefaultPredictor, im_list: List[ndarray]) -> List:
    with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
        inputs_list = []
        for original_image in im_list:
            # Apply pre-processing to image.
            if predictor.input_format == "RGB":
                # whether the model expects BGR inputs or RGB
                original_image = original_image[:, :, ::-1]
            height, width = original_image.shape[:2]
            # Do not apply original augmentation, which is resize.
            # image = predictor.aug.get_transform(original_image).apply_image(original_image)
            image = original_image
            image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
            inputs = {"image": image, "height": height, "width": width}
            inputs_list.append(inputs)
        predictions = predictor.model(inputs_list)
        return predictions

In [ ]:
len(dataset_dicts)

In [ ]:
for i in tqdm(range(ceil(len(dataset_dicts) / batch_size))):
    inds = list(range(batch_size * i, min(batch_size * (i + 1), len(dataset_dicts))))
    dataset_dicts_batch = [dataset_dicts[i] for i in inds]
    im_list = [cv2.imread(d["file_name"]) for d in dataset_dicts_batch]
    outputs_list = predict_batch(predictor, im_list)

    for im, outputs, d in zip(im_list, outputs_list, dataset_dicts_batch):
        resized_height, resized_width, ch = im.shape
        # outputs = predictor(im)
        if index < 5:
            # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
            v = Visualizer(
                im[:, :, ::-1],
                metadata=metadata,
                scale=0.5,
                instance_mode=ColorMode.IMAGE_BW
                # remove the colors of unsegmented pixels. This option is only available for segmentation models
            )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            # cv2_imshow(out.get_image()[:, :, ::-1])
            cv2.imwrite(str(outdir / f"pred_{index}.jpg"), out.get_image()[:, :, ::-1])

        image_id, dim0, dim1 = test_meta.iloc[index].values
        
        
        instances = outputs["instances"]
        if len(instances) == 0:
            # No finding, let's set 14 1 0 0 1 1x.
            result = {"image_id": image_id, "PredictionString": "14 1.0 0 0 1 1"}
        else:
            # Find some bbox...
            # print(f"index={index}, find {len(instances)} bbox.")
            fields: Dict[str, Any] = instances.get_fields()
            pred_classes = fields["pred_classes"]  # (n_boxes,)
            pred_scores = fields["scores"]
            # shape (n_boxes, 4). (xmin, ymin, xmax, ymax)
            pred_boxes = fields["pred_boxes"].tensor

            h_ratio = dim0 / resized_height
            w_ratio = dim1 / resized_width
            pred_boxes[:, [0, 2]] *= w_ratio
            pred_boxes[:, [1, 3]] *= h_ratio

            pred_classes_array = pred_classes.cpu().numpy()
            pred_boxes_array = pred_boxes.cpu().numpy()
            pred_scores_array = pred_scores.cpu().numpy()
            
            
            result = {
                "image_id": image_id,
                "PredictionString": format_pred(
                    pred_classes_array, pred_boxes_array, pred_scores_array
                ),
            }
        results_list.append(result)
        index += 1

In [ ]:
test_meta

In [ ]:
submission_det = pd.DataFrame(results_list, columns=['image_id', 'PredictionString'])
submission_det.to_csv(outdir/"submission_file10.csv", index=False)
submission_det

In [ ]:
submission_det.to_csv(outdir/"submission_file5.csv", index=False)


In [ ]:
submission_det.to_csv("submission_file2.csv", index=False)


In [ ]:
img_path = '../input/vinbigdata-chest-xray-resized-png-256x256/test/002a34c58c5b758217ed1f584ccbcfe9.png'
img = cv2.imread(img_path)
#shape of resized image
resized_h,resized_w, c = img.shape
print(img.shape)
output = predict(img)['instances']
img_path_dicom = "../input/vinbigdata-chest-xray-abnormalities-detection/test/002a34c58c5b758217ed1f584ccbcfe9.dicom"
img_dicom = pydicom.dcmread(img_path_dicom)
img_dicom = img_dicom.pixel_array
#shape of original image
org_h ,org_w = img_dicom.shape
print(img_dicom.shape)
h_ratio = resized_h/org_h
w_ratio = resized_w/org_w
print(h_ratio)
print(w_ratio)


In [ ]:
output

In [ ]:
field = output.get_fields()
pred_boxes = field["pred_boxes"].tensor

In [ ]:
pred_boxes

In [ ]:
pred_boxes

In [ ]:
pred_boxes[:, [0, 2]] *= w_ratio
pred_boxes[:, [1, 3]] *= h_ratio

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
    
evaluator = COCOEvaluator("vinbigdata_test", cfg, False, output_dir="./output")

In [ ]:
from detectron2.data import datasets, DatasetCatalog, MetadataCatalog, build_detection_train_loader, build_detection_test_loader


In [ ]:
val_loader = build_detection_test_loader(cfg, "vinbigdata_test") 

In [ ]:
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
test_img_dir = "../input/vinbigdata-chest-xray-resized-png-256x256/test"

In [ ]:
sub_df = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv")

In [ ]:
import pydicom

In [ ]:
img = pydicom.dcmread("../input/vinbigdata-chest-xray-abnormalities-detection/test/002a34c58c5b758217ed1f584ccbcfe9.dicom")
img =img.pixel_array
img.shape

In [ ]:
test_dicom_dir = "../input/vinbigdata-chest-xray-abnormalities-detection/test"

In [ ]:
def submit():
    for idx, row in tqdm(sub_df.iterrows(), total=len(sub_df)):
        
        img_path = os.path.join(test_img_dir, row.image_id + '.png')
        
        img = cv2.imread(img_path)
        img_path_dicom = os.path.join(test_dicom_dir,row.image_id + ".dicom")
        img_dicom = pydicom.dcmread(img_path_dicom)
        img_dicom = img_dicom.pixel_array
        org_h ,org_w = img_dicom.shape
        resized_h,resized_w, c = img.shape 
        h_ratio = resized_h/org_h
        w_ratio = resized_w/org_w
        outputs = predict(img)['instances']
        field = outputs.get_fields()
        pred_boxes = field["pred_boxes"].tensor
        pred_boxes[:, [0, 2]] *= w_ratio
        pred_boxes[:, [1, 3]] *= h_ratio
        labels = outputs.pred_classes.cpu().detach().numpy()
        boxes = [i.cpu().detach().numpy() for i in pred_boxes]
        scores = outputs.scores.cpu().detach().numpy()
        list_str = []
        for box, score,label in zip(boxes, scores, labels):
            
            box = list(map(int,box))
            score = round(score, 4)
            list_str.append(label)
            list_str.append(score)
            list_str.extend(box)
            sub_df.loc[idx, 'PredictionString'] = ' '.join(map(str, list_str))
        
    return sub_df

In [ ]:
img_path = "../input/vinbigdata-chest-xray-resized-png-256x256/test/002a34c58c5b758217ed1f584ccbcfe9.png"
img = cv2.imread(img_path)
img.shape

In [ ]:
sub_df = submit()
sub_df.to_csv('submission.csv', index=False)

In [ ]:
sub_df

In [ ]:
test_dicom_dir = "../input/vinbigdata-chest-xray-abnormalities-detection/test"

In [ ]:
for idx, row in tqdm(sub_df.iterrows(), total=len(sub_df)):
        img_path = os.path.join(test_img_dir, row.image_id + '.png')
        img_dicom_path = os.path.join(test_dicom_dir,row.image_id + '.dicom')
        img_dicom = pydicom.dcmread(img_dicom_path)
        img_dicom = img.pixel_array
        print(img.shape)
        #img = cv2.imread(img_path)
        

In [ ]:
list_str = []
list_str.append("14 1.0 0 0 1 1")
sub_df.loc[0, 'PredictionString'] = ' '.join(map(str, list_str))

In [ ]:
c = pd.read_csv("./results/debug/submission_file5.csv")

In [ ]:
c.to_csv('c.csv', index=False)

In [ ]:
if debug:
    dataset_dicts = dataset_dicts[:100]

results_list = []
index = 0
batch_size = 4

for i in tqdm(range(ceil(len(dataset_dicts) / batch_size))):
    inds = list(range(batch_size * i, min(batch_size * (i + 1), len(dataset_dicts))))
    dataset_dicts_batch = [dataset_dicts[i] for i in inds]
    im_list = [cv2.imread(d["file_name"]) for d in dataset_dicts_batch]
    outputs_list = predict_batch(predictor, im_list)

    for im, outputs, d in zip(im_list, outputs_list, dataset_dicts_batch):
        resized_height, resized_width, ch = im.shape
        # outputs = predictor(im)
        if index < 5:
            # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
            v = Visualizer(
                im[:, :, ::-1],
                metadata=metadata,
                scale=0.5,
                instance_mode=ColorMode.IMAGE_BW
                # remove the colors of unsegmented pixels. This option is only available for segmentation models
            )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            cv2.imwrite(str(outdir / f"pred_{index}.jpg"), out.get_image()[:, :, ::-1])
            plt.title(f"index {index}")
            plt.imshow(out.get_image()[:, :, ::-1])

        assert d["image_id"] == index
        image_id, dim0, dim1 = test_meta.iloc[index].values

        instances = outputs["instances"]
        if len(instances) == 0:
            # No finding, let's set 14 0 0 0 0.
            result = {"image_id": image_id, "PredictionString": "14 1.0 0 0 1 1"}
        else:
            # Find some bbox...
            # print(f"index={index}, find {len(instances)} bbox.")
            fields: Dict[str, Any] = instances.get_fields()
            pred_classes = fields["pred_classes"]  # (n_boxes,)
            pred_scores = fields["scores"]
            # shape (n_boxes, 4). (xmin, ymin, xmax, ymax)
            pred_boxes = fields["pred_boxes"].tensor

            h_ratio = dim0 / resized_height
            w_ratio = dim1 / resized_width
            pred_boxes[:, [0, 2]] *= w_ratio
            pred_boxes[:, [1, 3]] *= h_ratio
            pred_classes_array = pred_classes.cpu().numpy()
            pred_boxes_array = pred_boxes.cpu().numpy()
            pred_scores_array = pred_scores.cpu().numpy()

            result = {
                "image_id": image_id,
                "PredictionString": format_pred(
                    pred_classes_array, pred_boxes_array, pred_scores_array
                ),
            }
        results_list.append(result)
        index += 1